In [2]:
import pandas as pd
import numpy as np

In [11]:
df_quotes = pd.read_parquet('../data/quotes/GS_quotes_1s_2025-01-01_2025-01-31.parquet')

In [9]:
df_quotes

,ts,bid_price,ask_price,bid_size,ask_size,mid,spread,imbalance_l1,microprice,microprice_dev
0,2025-06-02 04:00:00-04:00,595.0,598.00,100.0,200.0,596.50,3.00,-0.333333,596.00,-0.5
1,2025-06-02 04:00:01-04:00,595.0,598.00,100.0,200.0,596.50,3.00,-0.333333,596.00,-0.5
2,2025-06-02 04:00:02-04:00,595.0,598.00,100.0,200.0,596.50,3.00,-0.333333,596.00,-0.5
3,2025-06-02 04:00:03-04:00,595.0,598.00,100.0,200.0,596.50,3.00,-0.333333,596.00,-0.5
4,2025-06-02 04:00:04-04:00,595.0,598.00,100.0,200.0,596.50,3.00,-0.333333,596.00,-0.5
...,...,...,...,...,...,...,...,...,...,...
57596,2025-06-02 19:59:56-04:00,598.0,598.72,100.0,100.0,598.36,0.72,0.000000,598.36,0.0
57597,2025-06-02 19:59:57-04:00,598.0,598.72,100.0,100.0,598.36,0.72,0.000000,598.36,0.0
57598,2025-06-02 19:59:58-04:00,598.0,598.72,100.0,100.0,598.36,0.72,0.000000,598.36,0.0
57599,2025-06-02 19:59:59-04:00,598.0,598.72,100.0,100.0,598.36,0.72,0.000000,598.36,0.0


In [12]:
df_prices = pd.read_parquet('../data/prices/GS_1s_2025-01-01_2025-01-31.parquet')
df_prices

,ts,symbol,open,high,low,close,volume,vwap,n_trades
0,2025-01-02 09:30:00-05:00,GS,NaN,NaN,NaN,NaN,0.0,NaN,0.0
1,2025-01-02 09:30:01-05:00,GS,579.3100,579.310,578.080,578.080,20990.0,579.2914,37.0
2,2025-01-02 09:30:02-05:00,GS,578.3250,578.325,578.325,578.325,846.0,578.5489,60.0
3,2025-01-02 09:30:03-05:00,GS,578.3250,578.325,578.325,578.325,231.0,578.4025,24.0
4,2025-01-02 09:30:04-05:00,GS,577.6800,578.330,577.440,578.110,2120.0,578.0346,38.0
...,...,...,...,...,...,...,...,...,...
467995,2025-01-31 15:59:55-05:00,GS,640.4800,640.480,640.480,640.480,319.0,640.4666,11.0
467996,2025-01-31 15:59:56-05:00,GS,640.3150,640.315,639.950,639.980,3306.0,640.0664,59.0
467997,2025-01-31 15:59:57-05:00,GS,640.0400,640.040,639.850,639.870,1436.0,639.9244,20.0
467998,2025-01-31 15:59:58-05:00,GS,640.1200,640.380,640.120,640.370,1292.0,640.2798,22.0


In [13]:
df_trades = pd.read_parquet('../data/trades/IVV_trades_1s_2025-06-01_2025-06-30.parquet')
df_trades

,ts,last,volume,n_trades,vwap,symbol
0,2025-06-02 09:30:00-04:00,590.585,6795.0,151,590.630425,IVV
1,2025-06-02 09:30:01-04:00,590.540,3177.0,608,590.652222,IVV
2,2025-06-02 09:30:02-04:00,590.880,2290.0,1774,590.630535,IVV
3,2025-06-02 09:30:03-04:00,590.540,638.0,232,590.772316,IVV
4,2025-06-02 09:30:04-04:00,590.540,164.0,89,590.761634,IVV
...,...,...,...,...,...,...
2183395,2025-06-27 15:59:55-04:00,617.740,25315.0,322,617.667493,IVV
2183396,2025-06-27 15:59:56-04:00,617.650,7769.0,95,617.659557,IVV
2183397,2025-06-27 15:59:57-04:00,617.620,12538.0,128,617.634216,IVV
2183398,2025-06-27 15:59:58-04:00,617.640,51746.0,487,617.555407,IVV
